In [1]:
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModels import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K



Using TensorFlow backend.


In [2]:
FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    print(input_shape)
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    #model.add(Dense(5, activation='relu', kernel_initializer='glorot_normal', name='fc3'))
    model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='tanh', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

In [3]:
#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride=20)
tModel.loadData()


Loading data for dataset 1


In [4]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*tModel.windowSize
shapeCNN1 = (tModel.windowSize, nFeatures, 1)
shapeDCNN1 = (tModel.windowSize, nFeatures)

modelRULSN = RULmodel_SN_1(shapeSN)
modelRULCNN = RULModel_CNN_1(shapeCNN1)
DCNN = RULCNNModel(shapeDCNN1[0], shapeDCNN1[1])

modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
modelRULCNN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
DCNN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#Add the models to the tunableModel object
tModel.addModel('RULmodel_SN_1', modelRULSN)
tModel.addModel('RULModel_CNN_1', modelRULCNN)
tModel.addModel('RULCNNModel', DCNN)

420
(30, 14, 1)


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C1", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:62: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C2", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glorot_normal", activation="tanh", name="C3", padding="same")`
/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), kernel_initializer="glo

In [5]:
tModel.printData(printTop=False)

Printing shapes
Training data (X, y)
(1817, 420)
(1817, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing last 5 elements
Training data (X, y)
[[ 0.64210526  0.27267231  0.08542941 ...,  0.11111111 -0.35042735
  -0.44458974]
 [ 0.11578947 -0.02599758  0.25900748 ..., -0.11111111 -0.33333333
  -0.29159856]
 [-0.01754386 -0.19770254  0.39225017 ...,  0.11111111 -0.48717949
  -0.03595946]
 [ 0.15789474  0.30592503  0.25039656 ...,  0.11111111 -0.69230769
  -0.60542661]
 [ 0.24912281  0.24183797  0.5001133  ...,  0.33333333 -0.77777778
  -0.63190585]]
[[ 40.]
 [ 30.]
 [ 20.]
 [ 10.]
 [  0.]]
Testing data (X, y)
[[-0.52280702 -0.58041112 -0.63154317 ..., -0.77777778  0.23076923
   0.70317097]
 [ 0.09473684 -0.16263603 -0.3242692  ...,  0.11111111 -0.36752137
   0.1627983 ]
 [-0.05964912  0.18681983 -0.28801269 ..., -0.11111111 -0.11111111
   0.37692056]
 [-0.35438596  0.00302297 -0.65012463 ..., -0.77777778  0.21367521
   0.37267081]
 [-0.23508772  0.10217654 -0.04373442 ...,  0.33333333 

In [5]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
tModel.setCurrentModel('RULmodel_SN_1')
tModel.getModelDescription()
tModel.trainCurrentModel(learningRateScheduler=lrate)


Description for current model: RULmodel_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 250)               105250    
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
fc2 (Dense)                  (None, 50)                12550     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
out (Dense)                  (None, 1)                 51        
Total params: 117,851
Trainable params: 117,851
Non-trainable params: 0
_________________________________________________________________
Epoch 1/250
939/939 [==============================] - 1s 742us/step - loss: 8372.1896 - me

Epoch 62/250
939/939 [==============================] - 0s 31us/step - loss: 344.9146 - mean_squared_error: 344.9146
Epoch 63/250
939/939 [==============================] - 0s 32us/step - loss: 347.4960 - mean_squared_error: 347.4960
Epoch 64/250
939/939 [==============================] - 0s 40us/step - loss: 379.1140 - mean_squared_error: 379.1140
Epoch 65/250
939/939 [==============================] - 0s 38us/step - loss: 343.8028 - mean_squared_error: 343.8028
Epoch 66/250
939/939 [==============================] - 0s 40us/step - loss: 360.2757 - mean_squared_error: 360.2757
Epoch 67/250
939/939 [==============================] - 0s 39us/step - loss: 354.1983 - mean_squared_error: 354.1983
Epoch 68/250
939/939 [==============================] - 0s 40us/step - loss: 367.2068 - mean_squared_error: 367.2068
Epoch 69/250
939/939 [==============================] - 0s 46us/step - loss: 363.4041 - mean_squared_error: 363.4041
Epoch 70/250
939/939 [==============================] - 0s 39us/

939/939 [==============================] - 0s 33us/step - loss: 298.7417 - mean_squared_error: 298.7417
Epoch 132/250
939/939 [==============================] - 0s 49us/step - loss: 273.4794 - mean_squared_error: 273.4794
Epoch 133/250
939/939 [==============================] - 0s 58us/step - loss: 303.3667 - mean_squared_error: 303.3667
Epoch 134/250
939/939 [==============================] - 0s 43us/step - loss: 298.8412 - mean_squared_error: 298.8412
Epoch 135/250
939/939 [==============================] - 0s 51us/step - loss: 280.9090 - mean_squared_error: 280.9090
Epoch 136/250
939/939 [==============================] - 0s 51us/step - loss: 273.6597 - mean_squared_error: 273.6597
Epoch 137/250
939/939 [==============================] - 0s 45us/step - loss: 299.8147 - mean_squared_error: 299.8147
Epoch 138/250
939/939 [==============================] - 0s 37us/step - loss: 276.2280 - mean_squared_error: 276.2280
Epoch 139/250
939/939 [==============================] - 0s 55us/step 

939/939 [==============================] - 0s 44us/step - loss: 246.3432 - mean_squared_error: 246.3432
Epoch 201/250
939/939 [==============================] - 0s 51us/step - loss: 257.6967 - mean_squared_error: 257.6967
Epoch 202/250
939/939 [==============================] - 0s 46us/step - loss: 264.7019 - mean_squared_error: 264.7019
Epoch 203/250
939/939 [==============================] - 0s 38us/step - loss: 240.6622 - mean_squared_error: 240.6622
Epoch 204/250
939/939 [==============================] - 0s 45us/step - loss: 253.8914 - mean_squared_error: 253.8914
Epoch 205/250
939/939 [==============================] - 0s 42us/step - loss: 240.5698 - mean_squared_error: 240.5698
Epoch 206/250
939/939 [==============================] - 0s 44us/step - loss: 261.1990 - mean_squared_error: 261.1990
Epoch 207/250
939/939 [==============================] - 0s 56us/step - loss: 236.9084 - mean_squared_error: 236.9084
Epoch 208/250
939/939 [==============================] - 0s 46us/step 

In [6]:
tModel.evaluateCurrentModel(["rhs"])
print("scores")

cScores = tModel.getModelScores(tModel.currentModelName)
rmse = math.sqrt(cScores['score_1'])
rhs = cScores['rhs']
time = tModel.getModelTimes(tModel.currentModelName)

print(cScores)
print("RMSE: {}".format(rmse))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 636us/step
scores
{'loss': 227.86319244384765, 'score_1': 227.86319244384765, 'rhs': 343.76446775051306}
RMSE: 15.09513803990701
RHS: 343.76446775051306
Time : 19.369804 seconds


In [7]:
print(custom_scores.compute_score("rhs", [1,2,3,4], [5,6,7,8]))

<class 'float'>
1.9672987905650814
